In [2]:
import pandas as pd 
import numpy as np
from datetime import date
import geopandas as gpd

In [10]:
domain = 'WY'

# SM location 
SMpath = '/nfs/attic/dfh/Aragon2/mar2020_snowmodel-dfhillWY_elev/'
incFile = SMpath+'code/snowmodel.inc'
#assim files
assimPath = '/nfs/attic/dfh/Aragon2/CSOassim/'+domain+'/'
codepath = SMpath+'code'
# load previously made file 
CSO_obs = '/nfs/attic/dfh/Aragon2/CSOassim/'+domain+'/CSO_assim_2018-2019_all.csv'
df = pd.read_csv(CSO_obs)

gdf = gpd.read_file(assimPath+"CSO_attributes.json", driver="GeoJSON")
# get cso doy 
gdf['dt'] = pd.to_datetime(gdf['timestamp'], format='%Y-%m-%dT%H:%M:%S')
gdf['dt'] = pd.to_datetime(gdf['dt']).dt.date
gdf['Y'] = pd.DatetimeIndex(gdf['dt']).year
gdf['swe']=df.swe

In [21]:
def make_SMassim_file(new,outFpath):
    f= open(outFpath,"w+")

    tot_obs=len(new)
    uq_day = np.unique(new.dt)
    num_days = len(uq_day)
    f.write('{:02.0f}\n'.format(num_days))
    for j in range(len(uq_day)):
        obs = new[new['dt']==uq_day[j]]
        d=new.D[new['dt']==uq_day[j]].values
        m=new.M[new['dt']==uq_day[j]].values
        y=new.Y[new['dt']==uq_day[j]].values
        date = str(y[0])+' '+str(m[0])+' '+str(d[0])
        obs_count = str(len(obs))
        f.write(date+' \n')
        f.write(obs_count+' \n')
        for k in range(len(obs)):
            ids = 100+k
            x= obs.x[obs.index[k]]
            y=obs.y[obs.index[k]]
            swe=obs.swe[obs.index[k]]
            f.write('{:3.0f}\t'.format(ids)+'{:10.0f}\t'.format(x)+'{:10.0f}\t'.format(y)+'{:3.2f}\n'.format(swe))
    f.close() 

In [ ]:
#function to edit SnowModel Files other than .par
def replace_line(file_name, line_num, text):
    lines = open(file_name, 'r').readlines()
    lines[line_num] = text
    out = open(file_name, 'w')
    out.writelines(lines)
    out.close()

In [ ]:
#var can be 'M', 'lc', 'aspect'
def SMassim_ensemble(df,var):
    today = date.today()
    hoy = today.strftime("%Y-%m-%d")
    path = assimPath+'assim'+hoy
    #create directory with initiation date for ensemble 
    !mkdir -p $path
    outFpath = SMpath+'swe_assim/swe_obs_test.dat'
    if var == 'elev':
        edges = np.histogram_bin_edges(df.dem_elev,bins=5, range=(df.dem_elev.min(),df.dem_elev.max()))
        print('edges:',edges)
        labs = np.arange(0,len(edges)-1,1)
        print('labels:',labs)
        bins = pd.cut(df['dem_elev'], edges,labels=labs)
        df['elev_bin']=bins    
        for i in range(len(labs)):
            new = df[df.elev_bin == i]
            make_SMassim_file(new,outFpath)
            #edit .inc file
            replace_line(incFile, 30, '      parameter (max_obs_dates='+str(len(new)+1)+')\n')
            #compile SM
            %cd $codepath
            ! ./compile_snowmodel.script
            #run snowmodel 
            %cd $SMpath
            ! ./snowmodel
            %cd  $assimPath
            #move swed.gdat file 
            oSWEpath = SMpath + 'outputs/wi_assim/swed.gdat'
            nSWEpath = path + '/elev_'+str(i)+'_swed.gdat'
            !mv $oSWEpath $nSWEpath    
    elif var == 'tc':
        edges = np.histogram_bin_edges(gdf.tc,bins=6, range=(gdf.tc.min(),gdf.tc.max()))
        print('edges:',edges)
        labs = np.arange(0,len(edges)-1,1)
        print('labels:',labs)
        bins = pd.cut(gdf['tc'], edges,labels=labs)
        gdf['tc_bin']=bins    
        for i in range(len(labs)):
            new = gdf[gdf.tc_bin == i]
            make_SMassim_file(new,outFpath)
            #edit .inc file
            replace_line(incFile, 30, '      parameter (max_obs_dates='+str(len(new)+1)+')\n')
            #compile SM
            %cd $codepath
            ! ./compile_snowmodel.script
            #run snowmodel 
            %cd $SMpath
            ! ./snowmodel
            %cd  $assimPath
            #move swed.gdat file 
            oSWEpath = SMpath + 'outputs/wi_assim/swed.gdat'
            nSWEpath = path + '/tc_'+str(i)+'_swed.gdat'
            !mv $oSWEpath $nSWEpath    
    else:
        uq = np.unique(df[var])
        for i in range(len(uq)):
            new = df[df[var] == uq[i]]
            make_SMassim_file(new,outFpath)
            #edit .inc file
            replace_line(incFile, 30, '      parameter (max_obs_dates='+str(len(new)+1)+')\n')
            #compile SM
            %cd $codepath
            ! ./compile_snowmodel.script
            #run snowmodel 
            %cd $SMpath
            ! ./snowmodel
            %cd  $assimPath
            #move swed.gdat file 
            oSWEpath = SMpath + 'outputs/wi_assim/swed.gdat'
            nSWEpath = path + '/'+var+'_'+str(uq[i])+'_swed.gdat'
            !mv $oSWEpath $nSWEpath    

In [ ]:
SMassim_ensemble(df,'tc')